<IMG SRC="https://github.com/jacquesroy/byte-size-data-science/raw/master/images/Banner.png" ALT="BSDS Banner" WIDTH=1195 HEIGHT=200>

<table align="left">
    <tr><td>
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a></td><td>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.</td>
    </tr>
    <tr><td>Jacques Roy, Byte Size Data Science</td><td> </td></tr>
    </table>

# Clustering
Simple use case trying to answer:
- Where are the accidents in the Chicago area?
- Where should we prioritize our efforts?

For this, we'll use multiple clustering algorithms.

In [ ]:
# youtube video related to this notebook
from IPython.display import IFrame

IFrame(src="https://www.youtube.com/embed/MhtuJfYNYdo?rel=0&amp;controls=0&amp;showinfo=0", width=560, height=315)

In [ ]:
# Library used to read datasets
# https://github.com/xmunoz/sodapy
!pip install sodapy 2>&1 >pipsodapy.txt

In [ ]:
# Install geopandas for geo objects support
!pip install geopandas 2>&1 >pipgeopandas.txt

In [ ]:
# Install folium for map rendering
!pip install folium 2>&1 >foliumpip.out

In [ ]:
# Libraries needed in the notebook
# import urllib3, requests, json
import pandas as pd
import numpy as np
# import io

# pd.set_option('display.max_colwidth', -1)

import matplotlib.pyplot as plt
# matplotlib.patches lets us create colored patches, which we can use for legends in plots
import matplotlib.patches as mpatches
%matplotlib inline

from sodapy import Socrata
import geopandas as gp
import folium

## Getting the data
We'll get the data dynamically using the socrata API. See also: <a href="https://github.com/jacquesroy/byte-size-data-science/blob/master/Notebooks/W005-FindingData.ipynb" target="_blank">W005-FindingData.ipyng</a>

To limit our dataset, we are reading the last 6 months of data.

The dataset attributes include:
```
  'alignment',                   'beat_of_occurrence',      'crash_date',           'crash_date_est_i',
  'crash_day_of_week',           'crash_hour',              'crash_month',          'crash_record_id',
  'crash_type',                  'damage',                  'date_police_notified', 'device_condition',
  'dooring_i',                   'first_crash_type',        'hit_and_run_i',        'injuries_fatal',
  'injuries_incapacitating',     'injuries_no_indication',
  'injuries_non_incapacitating', 'injuries_reported_not_evident',
  'injuries_total',              'injuries_unknown',        'intersection_related_i',
  'lane_cnt',                    'latitude',                'lighting_condition',   'location', 'longitude',
  'most_severe_injury',          'num_units',               'photos_taken_i',
  'posted_speed_limit',          'prim_contributory_cause', 'private_property_i',
  'rd_no',                       'report_type',             'road_defect',          'roadway_surface_cond',
  'sec_contributory_cause',      'statements_taken_i',      'street_direction',
  'street_name',                 'street_no',               'traffic_control_device', 'trafficway_type',
  'weather_condition',           'work_zone_i',             'work_zone_type',
  'workers_present_i'
```

In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

# If we wanted to do today:
# six_months = (date.today() - relativedelta(months=+6)).strftime('%Y-%m-%d')
# We are using a fix date for future comparisons
six_months = (date(2021,1,15) - relativedelta(months=+6)).strftime('%Y-%m-%d')
where = "crash_date > '{}'".format(six_months)

In [ ]:
# https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
crashes_df = pd.DataFrame(client.get("85ca-t3if", where=where, limit=10000))
offset = 10000
result = client.get("85ca-t3if", where=where, offset=offset, limit=10000)
while (len(result) > 0) :
    crashes_df = crashes_df.append(pd.DataFrame(result), sort=True)
    offset += 10000
    result = client.get("85ca-t3if", where=where, offset=offset, limit=10000)

print("Number of records: {}, number of columns: {}".format(crashes_df.shape[0], crashes_df.shape[1]))

## Selecting columns
In this example, we keep it simple and only work with the longitude and latitude.

When using clustering, we are not limited to positions such as longitude and latiutude.
We can iuse other attributes. For example, we can look at injuries and fatal injuries for example.

For the clustering to make sense, we need to make sure that all the values are normalized.
This way one value does not overwhelm the clustering. More on modeling later.

Here's what we need to do once we selected our columns:
- Reading the data from Socrata only gives us character strings, we need to convert to the proper types
- We also need to remove the records that don't have longitude and latitude.<br/>
This demonstrates that we always need to do data cleansing.

In [ ]:
# Has to be a better way to do this...
crashes_df = crashes_df[['injuries_fatal','injuries_total','latitude','longitude']]
# convert 'injuries_fatal' and 'injuries_total' to float otherwide, int causes problems.
crashes_df = crashes_df.astype({'injuries_fatal': float, 'injuries_total': float,
                                'latitude': float, 'longitude': float})
crashes_df = crashes_df.dropna()
crashes_df = crashes_df[crashes_df['longitude'] != 0]
crashes_df = crashes_df[crashes_df['latitude'] != 0]

#divide dataset into accident categories: fatal, non-fatal but with injuries, none of the above
killed_df = crashes_df[crashes_df['injuries_fatal']>0]
injured_df = crashes_df[np.logical_and(crashes_df['injuries_total']>0, crashes_df['injuries_fatal']==0)]
killed_or_injured_df = killed_df.append(injured_df)
nothing_df = crashes_df[np.logical_and(crashes_df['injuries_fatal']==0, crashes_df['injuries_total']==0)]

print("Number of records: {}".format(crashes_df.shape[0]))
print("Number of fatal accidents: {}".format(killed_df.shape[0]))
print("Number of injury accidents: {}".format(injured_df.shape[0]))
print("Number of no-injury accidents: {}".format(nothing_df.shape[0]))

crashes_df.describe()

## Scatterplot
Create a visualization of the accidents. Note that this is not a map.

Having a graphical representation of our data can give us some insights on how to proceed.

In [ ]:
# Use calculated values for the plot limits (border)
minlong = crashes_df['longitude'].min(axis=0) - 0.005
maxlong = crashes_df['longitude'].max(axis=0) + 0.005
minlat = crashes_df['latitude'].min(axis=0) - 0.005
maxlat = crashes_df['latitude'].max(axis=0) + 0.005
print("min, max, longitude, latitude: {}, {}, {}, {}".format(minlong,maxlong,minlat,maxlat))

In [ ]:
# I've also done subplots in notebook 61
nb_rows = 1
nb_plots = 2

fig, axes = plt.subplots(nrows=nb_rows, ncols=2)
fig.set_figheight(6)
fig.set_figwidth(18)

axes[0].scatter(crashes_df.longitude, crashes_df.latitude, color='darkseagreen', alpha=0.05, s=2)
axes[0].title.set_text('Motor Vehicle Accidents in Chicago (last six months)')
axes[0].set_xlabel('Longitude', labelpad = 5)
axes[0].set_ylabel('latitude', labelpad = 5)


axes[1].scatter(nothing_df.longitude, nothing_df.latitude, color='blue', alpha=0.04, s=2)
axes[1].scatter(injured_df.longitude, injured_df.latitude, color='yellow', alpha=0.12, s=2)
axes[1].scatter(killed_df.longitude, killed_df.latitude, color='red', alpha=1, s=2)

#create legend
blue_patch = mpatches.Patch( label='car body damage', color='blue')
yellow_patch = mpatches.Patch(color='yellow', label='personal injury')
red_patch = mpatches.Patch(color='red', label='lethal accidents')
axes[1].legend([blue_patch, yellow_patch, red_patch],('car body damage', 'personal injury', 'fatal accidents'), 
           loc='upper left', prop={'size':10})
axes[1].title.set_text('Severity of Motor Vehicle Collisions in Chicago')
axes[1].set_xlabel('Longitude', labelpad = 5)
axes[1].set_ylabel('latitude', labelpad = 5)
plt.show()


### What do we see?
- The Chicago area has a lot of accidents and they are all over the place.
- The first graph shows darket color in areas. This implies more accident density in some place.
- An overwhelming number of accident do not involve injurues or death.
- Reducing the opacity (alpha) of each point improved the information proivided by the map.


## Let's see the graph for only injuries and deaths

In [ ]:
# I've done subplots in notebook 61
nb_rows = 1
nb_plots = 2

fig, axes = plt.subplots(nrows=nb_rows, ncols=2)
fig.set_figheight(6)
fig.set_figwidth(18)

axes[0].scatter(crashes_df.longitude, crashes_df.latitude, alpha=0.05, s=4, color='darkseagreen')
axes[0].title.set_text('Motor Vehicle Accidents in Chicago (last six months)')
axes[0].set_xlabel('Longitude', labelpad = 5)
axes[0].set_ylabel('latitude', labelpad = 5)


axes[1].scatter(injured_df.longitude, injured_df.latitude, alpha=0.1, s=1, color='blue')
axes[1].scatter(killed_df.longitude, killed_df.latitude, color='red', s=5)

#create legend
blue_patch = mpatches.Patch( label='personal injuries', color='blue')
red_patch = mpatches.Patch(color='red', label='lethal accidents')
axes[1].legend([blue_patch, red_patch],('personal injuries', 'fatal accidents'), 
           loc='upper left', prop={'size':10})
axes[1].title.set_text('Severity of Motor Vehicle Collisions in Chicago')
axes[1].set_xlabel('Longitude', labelpad = 5)
axes[1].set_ylabel('latitude', labelpad = 5)
plt.show()


## Clustering with K-Means
Apply it only to accidents with fatalities for now.

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

distortions = []
K = range(1,60,1)
df = killed_df[['longitude','latitude']]
for k in K :
    kmeanModel = KMeans(n_clusters=k).fit(df)
    distortions.append(sum(np.min(cdist(df, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method for optimal k')
plt.show()

In [ ]:
# Pick 20 as the optimal K
# K Means Cluster
k=20
model = KMeans(n_clusters=k)
kmeans = model.fit(df)
vals=[0] * k
for i in kmeans.labels_ :
    vals[i] = vals[i] + 1
# Distribution between clusters
vals

## Show the clusters on a map

In [ ]:
data_pd = df.copy(deep=True)
data_pd['cgroup'] = kmeans.labels_
data_pd['cnt'] = [1] * kmeans.labels_.shape[0]

geo_gpd = gp.GeoDataFrame(data_pd, geometry=gp.points_from_xy(data_pd.longitude, data_pd.latitude))
# geo_gpd.head(5)

In [ ]:
# Create shapes for each cluster
group_gpd = geo_gpd[['cgroup','geometry','cnt']].dissolve(by='cgroup', aggfunc='sum').reset_index()
group2_gpd = gp.GeoDataFrame(group_gpd[['cgroup','cnt']],geometry=group_gpd.geometry.convex_hull)
# group2_gpd.head(5)

In [ ]:
latlong = geo_gpd[['latitude','longitude']].mean(axis=0)

chi_map = folium.Map(location=[latlong[0], latlong[1]], zoom_start=10, width="90%", height="90%")

for ix in range(group2_gpd['cnt'].count()) :
    folium.GeoJson(
        group2_gpd.iloc[ix]['geometry'],
        name="cluster-{0}".format(group2_gpd.iloc[ix]['cgroup']),
        tooltip="Cluster: {0}, count: {1}".format(group2_gpd.iloc[ix]['cgroup'],group2_gpd.iloc[ix]['cnt'] )
    ).add_to(chi_map)


folium.LayerControl().add_to(chi_map)
chi_map

### What do we learn?
The k-means clustering for fatal accidents are not that useful.<br/>
The number of accidents is too low to provide proper clustering. The clusters are too large for the number of accidents per cluster.

## Use K-Means with weights
Lets use both the fatal and injuries accidents.

We will weigh the fatalities higher than the injuries.

In [ ]:
df1 = killed_df[['longitude','latitude']].copy(deep=True)
df1['weight'] = [10] * df1.shape[0]
df2 = injured_df[['longitude','latitude']].copy(deep=True)
df2['weight'] = [1] * df2.shape[0]

df = df1.append(df2, ignore_index = True)
df.shape

In [ ]:
distortions = []
K = range(20,500,10)
for k in K :
    kmeanModel = KMeans(n_clusters=k).fit(df[['longitude','latitude']],sample_weight=df['weight'])
    distortions.append(sum(np.min(cdist(df[['longitude','latitude']], kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method for optimal k')
plt.show()

### More K-Means?
At think at this point, we are done with K-means. We need to look at other ways to cluster.

## Hierarchical
Let's try building the clusters from the bottom up on fatalities and injuries.

Even if we build the clusters from the bottom up, we can ask for a specific number of clusters (default 2).<br/>
If the number of clusters in None, we can get the entire hierarchy.

Additional parameters are avaialble to impact how clusters are merged together.

In [ ]:
# Import objects assuming the k-means section was skipped
from scipy import ndimage 
from scipy.cluster import hierarchy 
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial import distance_matrix 
from matplotlib import pyplot as plt 
from sklearn import manifold 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets.samples_generator import make_blobs

In [ ]:
ac = AgglomerativeClustering(n_clusters=None,distance_threshold=0)
clusters = ac.fit(df[['longitude','latitude']])

In [ ]:
def get_linkage_matrix(model, **kwargs):
    # Create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)
    return(linkage_matrix)

In [ ]:
linkage_matrix = get_linkage_matrix(clusters)

In [ ]:
plt.figure(figsize=(9, 9))  
ret = dendrogram(linkage_matrix, truncate_mode='lastp', p=50, no_plot=False, orientation='left')

### Dendrogram
In this figure (Dendogram) we see up to 50 clusters with the number of accidents in each.

We could pick different numbers of clusters and display on a map to see if we get additional insights.
Instead, we move on another typoe of clustering that may be more promising.

### DBSCAN
Density-based spatial clustering.
Locates regions of high density that are separated from one another by regions of low density.

info:

- https://scikit-learn.org/stable/modules/clustering.html#clustering
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster
- https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html
- https://hdbscan.readthedocs.io/en/latest/comparing_clustering_algorithms.html

The key statement in the model creation below is:

```
db = DBSCAN(eps=0.00015, min_samples=30).fit(crashes_df[['latitude','longitude']])

```
The parameters we really have to pay attention to are:
- `eps` (default 0.5): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
- `min_samples` (default 5): The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
- `metric` (default 'euclidean'): How to calculate the distance between points.

The parameter that needs additional explanation is `eps`. How did we get to 0.001 ?

The key is to consider that we are dealing with locations, and longitudes and latitudes.
The distance has to be in the proper scale. In this example, we are looking for accident hotspots.
They should be at an intersection or between two intersections.
Luckily for us, we don't need to be too accurate.


The assumption is that is is likely at an intersection. How large should we consider this intersection?

A street is roughly 30 feet wide. If the accident is in the middle of the intersection, we should 
be ok using a distance of around 40 feet.

In the Chicago area, the value 0.00015 represents roughly:
- Horizontal (longitudinal) distance: 40 feet
- Vertical (latitudinal) distance: 54 feet
- Diagonal distance: 68 feet

The earth is not quite round. Take a look at <a href="https://youtu.be/LKANJBxxtuQ" target="_blank">video 45</a> for more information.


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [ ]:
# Should we aim for a number of clusters that is lower than the number of accidents with fatalities (<92)?
# data_np = crashes_df[['latitude','longitude']].to_numpy()

# eps=0.001 represents roughly 
db = DBSCAN(eps=0.00015, min_samples=18).fit(crashes_df[['latitude','longitude']])

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)


## Display the cluster centers on a map
Note that we display only the top 20 by selecting them from the list.

In [ ]:
results_df = crashes_df[['latitude','longitude']].copy(deep=True)
results_df['cluster'] = db.labels_
results_df = results_df[results_df['cluster'] > -1].reset_index(drop=True) # remove noise points
results_df = results_df.groupby('cluster').agg(latitude=pd.NamedAgg(column='latitude',aggfunc="mean"), 
                                               longitude = pd.NamedAgg(column='longitude',aggfunc="mean"), 
                                               cnt = pd.NamedAgg(column='cluster', aggfunc='count') ).\
                                           reset_index().sort_values('cnt', ascending=False).reset_index(drop=True)

minval=results_df['cnt'].min().astype(int).item()
maxval= 1 + results_df['cnt'].max().astype(int).item()
# print("minval: {}, maxval: {}".format(minval,maxval))
# results_df.head()

In [ ]:
# Use a colormap to colorcode the count of accidents in each cluster
# It's difficult to find a good colormap to use.
# see: https://matplotlib.org/3.3.3/gallery/color/colormap_reference.html
import matplotlib.cm as cm
import numpy as np

colors = cm.get_cmap('viridis', maxval)(range(maxval),bytes=True)

rgbcolors = []
for v in colors :
    col = np.floor(v * 255)
    r = int(col[0])
    g = int(col[1])
    b = int(col[2])
    rgbcolors.append('#' + '{0:#08x}'.format(((r * 65536) + (g * 256) + b))[2:])

In [ ]:
# Display the average center of each group
latlong = results_df[['latitude','longitude']].mean(axis=0) # To center the map
chi_map = folium.Map(location=[latlong[0], latlong[1]], zoom_start=10, width="90%", height="90%")

# Since the dataframe is ordered, we can display the top 20 only
for idx, coord in results_df[0:20].iterrows():
    tooltip_content="Cluster: {0}, count: {1}".format(coord['cluster'].astype(int),coord['cnt'].astype(int) )
    folium.Circle(radius=20,
                  location=[coord['latitude'], coord['longitude']],
                  # popup=row.hgroup,
                  color=rgbcolors[coord['cnt'].astype(int) ],
                  tooltip=tooltip_content,
                  fill=True,
                  fill_color=rgbcolors[coord['cnt'].astype(int) ]
    ).add_to(chi_map)
    
chi_map

## What next?
We have identified the locations where there are at least 10 accidents based on the last 6 months of data.<br/>
We displayed the top 20 cluster locations.

We can drill down with the map to see what is around that location.